In [1]:
## Knight line
## Start with a single stack. Move part of a stack like a knight, leaving at least one piece behind. If you make a row of four (orthogonally or diagonally), you win. All pieces must remain connected to all other pieces after each move.

In [2]:
import games
import copy
import scipy.special
import utils
import search
import numpy as np
import random
import time
from operator import itemgetter

In [3]:
SIZE = 13 ## size of the board, n*n
KNIGHTS = 30 ## number of knights a player starts with, n*2 in total
DX = [-2,-1,1,2,2,1,-1,-2] ## possible moves for a knight from chess
DY = [1,2,2,1,-1,-2,-2,-1] ## actual position -2 horizontal +1 vertical = L shape, knight has 8 different places he can go to

In [4]:
## this function checks for a player's all possible moves, [[xfrom,yfrom],[xto,yto],number_of_knights]
def possible(places, player):
    
    ## checks if the found place on the grid is empty or not, if not player cannot move there
    def isempty(x,y,knights,player):
        if [x,y,knights,player] in places:
            return True
        return False
    
    ## main algorithm, that goes over all places on the board and checks if a certain move is valid, [[5,5],[6,8],15] and [[5,5],[6,8],5] are NOT the same moves!!!
    def check(squares):
        reach = []
        for p in squares:
            x,y,knights,playerOnPlace = p
            for dx,dy in zip(DX,DY):
                if neighbours(x+dx,y+dy) and isempty(x+dx,y+dy,0,'.'):
                    for k in range(1,knights):
                        reach.append([[x,y],[x+dx,y+dy],k,playerOnPlace])
        return reach
    
    ## checks if a valid space has neighbours around it, if not we cannot move there
    def neighbours(x,y):
        adjacency = [[i,j] for i in (-1,0,1) for j in (-1,0,1) if not (i == j == 0)]
        for dx, dy in adjacency:
            if 1 <= (x+dx) <= SIZE and 0 <= (y+dy) <=SIZE:
                for place in places:
                    if place[2] > 0 and place[0]==x+dx and place[1]==y+dy:
                        return True
        return False
    
    ## fill up a list of places that the player has knights on, here we also check if the number is > 1, if not the player cannot move
    possible = []
    whiteSquares = []
    blackSquares = []
    for p in places:
        x,y,knights,playerOnPlace = p
        if playerOnPlace == player:
            if knights > 1:
                if player == 'X':
                    whiteSquares.append(p)
                else:
                    blackSquares.append(p)
    if len(whiteSquares)==0:
        possible = check(blackSquares)
    else:
        possible = check(whiteSquares)
    
    return possible

In [5]:
## the game class
class KnightLine(games.Game):
    
    ## setting up initial state and moves for the X player, the shuffle makes the more interesting
    def __init__(self, places):
        board = places.copy()
        moves = possible(board, 'X')
        random.shuffle(moves)
        self.initial = games.GameState(to_move='X', utility=0, board=board, moves=moves)
    
    ## returns the possible moves in the current state
    def actions(self, state):
        return state.moves
    
    ## result of an action performed on the state
    def result(self, state, move):
        
        ## not necessary
        if move not in state.moves:
            return state
        
        ## apply the move, remove the knights from old place and add them to the new place, same with player symbol
        fr,to,howMany,player = move
        xfr,yfr = fr
        xto,yto = to
        board = state.board.copy()
        for p in board:
            if xfr == p[0] and yfr == p[1]:
                p[2]-=howMany
            if xto == p[0] and yto == p[1]:
                p[2]+=howMany
                p[3] = player
        
        ## calculate moves for the next player
        moves = possible(board,'X' if player=='O' else 'O')
        random.shuffle(moves)
        
        ## clear state moves, this was an issue while using minmax search, unnecessary moves are removed from memory
        state.moves.clear()
        
        ## apply the new game state, switch player, compute current state utility
        return games.GameState(to_move=('O' if state.to_move=='X' else 'X'),
                         utility = self.compute_utility(moves, board, state.to_move),
                         board=board, moves=moves)
    
    ## positive for X negative for O
    def utility(self, state, player):
        return state.utility if player=='X' else -state.utility

    ## if there are no moves left in this state, or utility is lower or higher than 4, which is a WIN or a LOSE return True
    def terminal_test(self, state):
        return len(state.moves)==0 or state.utility < -4 or state.utility > 4

    ## displays the game state, uses 2 sorts by key values for lists of list
    def display(self, state):
        board = state.board
        board = sorted(board, key=itemgetter(1))
        board = sorted(board, key=itemgetter(0))
        i = 1
        for p in board:
            printable = str(str(p[3])+str(p[2]))
            print(printable.ljust(5), end=' ')
            if(i%SIZE == 0):
                print()
            i+=1
    
    ## computes the utility of the current state
    def compute_utility(self, moves, board, player):
        
        ## checks for consecutive knights, return the maximum consecutive number
        def check_consecutive(l,player):
            found = []
            i = 0
            while i < len(l):
                if l[i]==player:
                    tmpblack = 0
                    for j in range(i,len(l)):
                        if l[j]!=player:
                            break
                        tmpblack += 1
                    found.append(tmpblack)
                    i = j
                i += 1
            return max(found)
        
        def k_in_row(move, board):
            x,y = move[1]
            player = move[3]
            removeXB,removeXU,removeYB,removeYU = 4,4,4,4
            
            ## we check for diagonality in 2 diagonal rows, one from left top to right bottom, one from right top to left bottom
            for p in board:
                if x==p[0] and y==p[1]:
                    for l in range(1,5):
                        row = []
                        for i,j in zip(range(x-removeXB+l,x+l),range(y-removeYB+l, y+l)):
                            if p[0]==i and p[1]==j:
                                row.append(player)
                            else:
                                for pb in board:
                                    if pb[0]==i and pb[1]==j:
                                        row.append(pb[3])
                        ## first we check how many knights will be in diagonal with the new move and calculate a weighted utility using the binomial coefficients
                        ## I used a heuristic where a move with one knight only would be equally as bad as using all of your knights
                        ## from the games I played it seemed like the best strategy to win is to develop the board as much as you can
                        ## this way the best move without counting the diagonal weights is to move approx half of your knights
                        ## using this heuristic you can easily defend and develop a position
                        ## the coefficient is weighted so its values range from 0 to 1
                        ## we then add how many knights we would have in diagonal (1,2,3 or 4)
                        ## we will always have at least one, that will be our current move if it happened
                        ## so winning usually requires a number greater than 4
                        numbers.append(check_consecutive(row, player)+((scipy.special.binom(maxTake,move[2]) - scipy.special.binom(KNIGHTS-1,KNIGHTS-1))/(scipy.special.binom(KNIGHTS-1,(maxTake+1)/2)-scipy.special.binom(KNIGHTS-1,KNIGHTS-1))))

                    for l in range(1,5):
                        row = []
                        for i,j in zip(range(x-removeXB+l,x+l),range(y+removeYU-l, y-l,-1)):
                            if p[0]==i and p[1]==j:
                                row.append(player)
                            else:
                                for pb in board:
                                    if pb[0]==i and pb[1]==j:
                                        row.append(pb[3])
                        numbers.append(check_consecutive(row, player)+((scipy.special.binom(maxTake,move[2]) - scipy.special.binom(KNIGHTS-1,KNIGHTS-1))/(scipy.special.binom(KNIGHTS-1,(maxTake+1)/2)-scipy.special.binom(KNIGHTS-1,KNIGHTS-1))))
        
        ## some variables
        numbers = []
        maxTake = 0
        
        ## if there no moves left, its a tie so a 0 is returned
        if not moves:
            return 0
        
        ## there was no other way around this, we need the BIGGEST (eg. if player has 20 on a place he can move max 19) amount the player can move at once
        moves = sorted(moves, key=itemgetter(2))
        moves = moves[::-1]
        maxTake = moves[0][2]
        
        ## for each move check for diagonality and compute a new utility measurement
        for m in moves:
            if(m[3]=='X' if player=='O' else 'O'):
                k_in_row(m, board)
    
        return -max(numbers) if player == 'X' else max(numbers)
    
    ## taken from AIMA code, extended it with deep copying the state before the search then printing out the move chosen
    def play_game(self, *players):
        state = self.initial
        while True:
            for player in players:
                tmpstate = copy.deepcopy(state)
                move = player(self, tmpstate)
                #print(move)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

In [6]:
## AIMA body, just played with the depth
def alpha_beta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    
    player = game.to_move(state)
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state: game.utility(state, player))
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [7]:
## AIMA body
def minmax_decision(state, game):

    player = game.to_move(state)
    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    ret = max(game.actions(state), key=lambda a: min_value(game.result(state, a)))
    return ret

In [8]:
## AIMA 
def alpha_beta_player(game, state):
    return alpha_beta_cutoff_search(state, game)

def minmax_player(game,state):
    return minmax_decision(state,game)

def random_player(game, state):
    a = list(game.actions(state))
    if a:
        return random.choice(list(game.actions(state)))
    else:
        return None

In [9]:
## make a list of grid places
## coordinates, how many knights on the place, who is placed on that place
## on a 13x13 grid when we arrive at i,j = 6,7 or 7,7 we add the two players
places13 = []
for i in range (1,SIZE+1):
    for j in range (1,SIZE+1):
        if i==(SIZE-1)/2 and j==(SIZE+1)/2:
            c=[i,j,KNIGHTS,'O']
        elif i==(SIZE+1)/2 and j==(SIZE+1)/2:
            c=[i,j,KNIGHTS,'X']
        else:
            c = [i,j,0,'.']
        places13.append(c)


## playing the game
##rand = KnightLine(places13)
##time1 = time.time()
##print(rand.play_game(random_player,random_player))

minmax = KnightLine(places13)
time2 = time.time()
print(minmax.play_game(minmax_player,minmax_player))

##alpha_beta = KnightLine(places13)
##time3 = time.time()
##print(alpha_beta.play_game(alpha_beta_player,alpha_beta_player))

##print('Random Playing time was: {:.3f}'.format(time.time()-time1))
print('MinMax Playing time was: {:.3f}'.format(time.time()-time2))
##print('Alpha-Beta Playing time was: {:.3f}'.format(time.time()-time3))

.0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    O1    .0    .0    O3    .0    .0    .0    .0    .0    .0    
.0    .0    .0    O8    .0    O1    X1    .0    .0    .0    .0    .0    .0    
.0    .0    .0    X1    O1    X1    .0    O1    .0    .0    .0    .0    .0    
.0    .0    O1    X6    X1    .0    O1    X3    .0    .0    .0    .0    .0    
.0    .0    X1    O3    O8    X2    X3    .0    O2    .0    .0    .0    .0    
.0    .0    .0    X2    X9    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    .0    
.0    .0    .0    .0    .0    .0    .0    .0    .0  